In [1]:
#Libraries
import pandas as pd
import re
import requests

import os
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import config 

from getpass import getpass

import random

from selenium.webdriver.common.action_chains import ActionChains

In [2]:
## first we need to initiate the driver##
driver = webdriver.Chrome('C:\\Users\\catar\\OneDrive\\Documentos\\chromedriver.exe')

In [3]:
# open the website
driver.get('https://www.glassdoor.com')

In [4]:
# Login in
signin = driver.find_elements_by_css_selector("button[class^='d-flex align-items-center justify-content-center order-1 order-md-2 mr-auto mr-md-0 p-0 LockedHomeHeaderStyles__signInButton']")[0].click()

time.sleep(2.5) ## sleep timer in order to mimic human interaction

In [5]:
# Insert email
email = getpass('Enter your email') ## input email

email_box = driver.find_element_by_id("userEmail") ## box to input email

email_box.send_keys(email)  ## send email to webpage

time.sleep(3.5) ## sleep timer in order to mimic human interaction

In [6]:
#Insert password

password = getpass('Enter your password') ## input password

pass_box = driver.find_element_by_id('userPassword') ## box to input password

pass_box.send_keys(password)  ## send password to webpage

time.sleep(1.3) ## sleep timer in order to mimic human interaction

In [7]:
#login

login = driver.find_elements_by_css_selector("button[class^='gd-ui-button minWidthBtn css-8i7bc2']")[0].click()

time.sleep(1.6)  ## sleep timer in order to mimic human interaction


In [8]:
#######################################

In [9]:
# Lets try a function for that

def interview_info(job_title,location):
    
    actions = ActionChains(driver)
    
    # first lets go to interview page
    interview_icon = "//a[@data-test = 'site-header-interviews']" 
    interview_button = driver.find_element_by_xpath(interview_icon)
    interview_click = interview_button.click()
    time.sleep(random.uniform(5,7)) ## sleep timer in order to mimic human interaction
    
    # lets input the job title to search
    job_title_box = driver.find_element_by_id("KeywordSearch") ## box to input job title
    job_title_box.send_keys(job_title) ## send job title to page
    time.sleep(random.uniform(5,7)) ## sleep timer in order to mimic human interaction
    
    # lets input the location to search
    location_box = driver.find_element_by_id("LocationSearch")  ## box to input job title
    location_box.clear() ##clear initial input
    location_box.send_keys(location) ## send job title to page
    time.sleep(random.uniform(5,7)) ## sleep timer in order to mimic human interaction
    
    # find information
    interview_search = driver.find_element_by_id('HeroSearchButton') ## click search button
    interview_search.click()  ## click search button
    time.sleep(random.uniform(5,7))  ## sleep timer in order to mimic human interaction
    
    ## Now that we have the results lets check total number of results and page of results
    results = "//div[@class='reviewCount center']"
    total_results = driver.find_element_by_xpath(results).text
    number_of_results = int(total_results)
    number_of_pages = round(number_of_results/10)

    ## Create to add the info scrapped
    job_company = []
    questions = []
    answers = []

    ## Actions
    actions = ActionChains(driver)
    
    #lets iterate throw the questions
    for i in range(1,(number_of_results+1)):
        display(i)
        
        #company info
        xpath1 = "//div[starts-with(@id, 'InterviewQuestionResult_"+str(i)+"')]//span[@class='authorInfo']"
        job_company_name = driver.find_element_by_xpath(xpath1)
        job_company_name_text = job_company_name.text
        display(job_company_name_text)
        job_company.append(job_company_name_text)
        time.sleep(random.uniform(3,6))

        #question info
        xpath2 = "//div[starts-with(@id, 'InterviewQuestionResult_"+str(i)+"')]//p[@class='questionText h3']"
        questions_gd = driver.find_element_by_xpath(xpath2)
        questions_gd_text = questions_gd.text
        display(questions_gd_text)
        questions.append(questions_gd_text)
        time.sleep(random.uniform(3,6))

        #answer info
        try:
            #click answer button
            if ((i%10) != 1):   
                xpath3 = "//div[starts-with(@id, 'InterviewQuestionResult_"+str(i)+"')]//a[@class='userResponseLink margTop block hiddenLink mmLink ']"
                answer_icon = driver.find_element_by_xpath(xpath3)
                answer_icon_click = answer_icon.click()
                time.sleep(random.uniform(3,6))         
        
            #get answer info            
            xpath4 = "//div[starts-with(@id, 'InterviewQuestionResult_"+str(i)+"')]//div[@class='responseText padTopSm tbl fill']"
            answers_gd = driver.find_element_by_xpath(xpath4)
            answers_gd_text = answers_gd.text
            display(answers_gd_text)
            answers.append(answers_gd_text)
            time.sleep(random.uniform(3,6))
        
        #if no answer presented
        except:
            answers_gd_text = 'No answer'
            display(answers_gd_text)
            answers.append(answers_gd_text)
            time.sleep(random.uniform(3,6))
            
        #go to next page
        if ((i%10) == 0):
            xpath_page = "//div[@class='pagingControls cell middle']//li[@class ='next']"
            page_change = driver.find_element_by_xpath(xpath_page)
            page_change_button = page_change.click()
            time.sleep(random.uniform(35,40))
    
    
    #lets retrieve dataframe with this info
    interview_data = pd.DataFrame([job_company,questions,answers]).T
    interview_data.columns = ['job_company', 'questions','answers']
    display(interview_data)

    return interview_data

In [10]:
def data_to_csv(scrap_data):
    t = time.localtime() # check time
    m = str(t.tm_mon) #scrap month
    d = str(t.tm_mday) #scrap day
    h = str(t.tm_hour) #scrap hour
    mt = str(t.tm_min) #scrap minute

    #lets structure how to present this info in the file name
    structure = "_m"+str(m)+"_d"+str(d)+"_h"+str(h)+"_"+str(mt)

    #turn into csv
    scrap_data.to_csv("interview_data"+structure+".csv",sep = ',',index=False)


In [11]:
job_title_da = 'Data Analyst' ## Lets search Data analytics
location_pt = 'Portugal' ## Lets search Data analytics

data_to_csv(interview_info(job_title_da,location_pt))

1

'Senior Data Scientist at OLX Group was asked...'

'explain the reasoning in resolving the case'

'I answer all the insights I had based on data analysis and business acumen'

2

...

66

'Junior Data Scientist at FARFETCH was asked...'

"They asked me to explain the advantages of my case-study solution to a business person, who don't know anything about to Data Science and Machine Learning."

'No answer'

,job_company,questions,answers
0,Senior Data Scientist at OLX Group was asked...,explain the reasoning in resolving the case,I answer all the insights I had based on data ...
1,Data Science at MOG Technologies was asked...,"Nothing too technical, they presented one of t...",I told them that it could be solved and presen...
2,Data Scientist at Smart Consulting was asked...,Tell me about 2 projects that you seem fit to ...,Described two projetcs about AI
3,Data Scientist at Deloitte was asked...,What your motivations are?,"If the data adequately represent reality, by s..."
4,Data Scientist at 360imprimir was asked...,Logic question( cases resolution),Detailed explanation about the reasoning behin...
...,...,...,...
61,Data Scientist at Builtrix was asked...,Questions related to your past work experience...,No answer
62,Data Scientist at Feedzai was asked...,"that is quite fuzzy screening test, with neces...",No answer
63,Data Scientist at Feedzai was asked...,Describe two of your past projects that had Ma...,No answer
64,Data Scientist at Miniclip was asked...,- Technical questions about the challenge,No answer


In [12]:
job_title_da = 'Web Developer' ## Lets search Data analytics
location_pt = 'Portugal' ## Lets search Data analytics

data_to_csv(interview_info(job_title_da,location_pt))

1

'Web Developer Junior at Prime IT was asked...'

'Seu eu sabia o que fazia a Prime IT'

'Falando o que havia lido a respeito da empresa e de seus metodos de trabalho'

2

...

31

'Full Stack Web Developer at ITSector was asked...'

'On a blank board do a simple algorithm to find a capicua.'

'No answer'

,job_company,questions,answers
0,Web Developer Junior at Prime IT was asked...,Seu eu sabia o que fazia a Prime IT,Falando o que havia lido a respeito da empresa...
1,Web Developer at Leya was asked...,They asked all the usual job interview questio...,"I wrote down the general programming logic, in..."
2,Web Developer at Landing.jobs was asked...,If I would feel Ok with unpaid extra time ever...,Obviously not.
3,Junior Web Developer at iTGROW was asked...,Difference between foreach and map Java functions,"Map returns values, foreach doesn't,..."
4,Web Developer at PDM&FC was asked...,I was asked about Java and JavaScript even tho...,No answer
5,Web Developer at Wiremaze was asked...,What's your salary expectation?,No answer
6,Web Developer at Eticadata was asked...,"Database questions, CRUD",No answer
7,Web Developer at rVisual was asked...,they asked me to fix old broken code without b...,No answer
8,Web Developer at Wiremaze was asked...,"I was asked to do a simple HTML page, with som...",No answer
9,Web Developer at Alter Solutions was asked...,What kind of projects that you work?,No answer


In [13]:
job_title_da = 'UX Designer' ## Lets search Data analytics
location_pt = 'Portugal' ## Lets search Data analytics

data_to_csv(interview_info(job_title_da,location_pt))

1

'UX Designer at Sport Zone was asked...'

'Can you work under pressure?'

'Yes.'

2

...

11

'UI/UX Designer at OutSystems was asked...'

'Describe the UX process you are aware of.'

'No answer'

,job_company,questions,answers
0,UX Designer at Sport Zone was asked...,Can you work under pressure?,Yes.
1,UX Designer at Sport Zone was asked...,Why do you want to leave you actual job?,I want a new experience.
2,UX Designer at Sport Zone was asked...,Talk about you and your experience.,I just focused on my experience.
3,UI & UX Designer at BNP Paribas was asked...,Previous experiences with data visualization,No answer
4,UX Designer at Zalando was asked...,"The interview was more about my experience, me...",No answer
5,UI/UX Designer at Pixelmatters was asked...,What is for you/your understading of UI/UX Des...,No answer
6,Ui\Ux Designer at YGroup Companies was asked...,"If you had to validate this solution, how woul...",No answer
7,UX Designer at Sport Zone was asked...,What do you expect from this job?,Be happy doing what I love to do. A fair salar...
8,UX Designer at Feedzai was asked...,They questioned Front-End Development instead ...,No answer
9,Sr UX Designer 2 at Springer Nature was asked...,"Tell me about your experience, explain your de...",No answer


In [14]:
job_title_da = 'UI Designer' ## Lets search Data analytics
location_pt = 'Portugal' ## Lets search Data analytics

data_to_csv(interview_info(job_title_da,location_pt))

1

'UI Designer at WIT Software was asked...'

'Why do you want to join us? What motivates you?'

'To live and learn, to contribute and to evolve, because what motivates me to do this, my children.'

2

...

8

'UI/UX Designer at OutSystems was asked...'

'Describe the UX process you are aware of.'

'No answer'

,job_company,questions,answers
0,UI Designer at WIT Software was asked...,Why do you want to join us? What motivates you?,"To live and learn, to contribute and to evolve..."
1,UX-UI Designer at NDrive was asked...,- Why were i interested in working there and h...,No answer
2,UI & UX Designer at BNP Paribas was asked...,Previous experiences with data visualization,No answer
3,UI/UX Designer at Pixelmatters was asked...,What is for you/your understading of UI/UX Des...,No answer
4,Ui\Ux Designer at YGroup Companies was asked...,"If you had to validate this solution, how woul...",No answer
5,UX-UI Designer at BOOST IT was asked...,The interview was mostly a Q&A session based o...,No answer
6,User Experience/UI Designer at Blip was asked...,"Regular questions about your method, collabora...",No answer
7,UI/UX Designer at OutSystems was asked...,Describe the UX process you are aware of.,No answer


In [51]:
driver.close()